<a href="https://colab.research.google.com/github/akanksha0911/DE_ETLpipeline/blob/main/PS_DE_LOCAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the libraries

In [ ]:
import pandas as pd
import re
from pathlib import Path
import uuid
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_datetime64_any_dtype

In [ ]:
# Read the first sheet into a DataFrame
LEH_df = pd.read_excel('/content/Homework_Assignment_Data.xlsx', sheet_name='Loyalty Earned Hourly')

# Read the second sheet into a DataFrame
purchases_df = pd.read_excel('/content/Homework_Assignment_Data.xlsx', sheet_name='Purchases')


In [ ]:
LEH_df

,date,userId,country,total_lp_earned
0,2022-04-01 20:00:00,WW42LKF,CA,69.622583
1,2022-04-01 01:00:00,TT57LKF,US,5.312675
2,2022-04-03 00:00:00,WW42LKF,US,24.911172
3,2022-04-03 08:00:00,TT57LKF,US,1.617805
4,2022-04-02 21:00:00,WW42LKF,US,11.168712
5,2022-04-01 04:00:00,WW42LKF,US,77.899582
6,2022-04-01 20:00:00,AA22LKF,US,26.025999
7,2022-04-02 17:00:00,AA22LKF,CA,52.235656
8,2022-04-02 05:00:00,WW42LKF,US,10.073606
9,2022-04-02 18:00:00,TT57LKF,US,33.204787


In [ ]:
purchases_df

,date,userId,revenue,transaction_id
0,2022-04-01 22:53:05,WW42LKF,PriceInUSD=4.99,2e6fb51b-ff50-4aa0-ba34-369f2c139b33
1,2022-04-01 22:53:05,WW42LKF,PriceInUSD=4.99,2e6fb51b-ff50-4aa0-ba34-369f2c139b33
2,2022-04-01 19:03:22,TT57LKF,PriceInUSD=1.99,0ca446d6-4a7d-4778-bc03-45d470e157a1
3,2022-04-03 17:17:38,WW42LKF,PriceInUSD=4.99,e61df479-4b96-44a6-ae0c-4d49fad6e368
4,2022-04-02 11:53:15,TT57LKF,PriceInUSD=4.99,51d6800b-7b75-42be-a7a1-d5e8b11d7f7d
5,2022-04-03 10:30:03,WW42LKF,PriceInUSD=4.99,f19cfd04-eaef-4ae9-899e-92781fadad99
6,2022-04-02 05:25:54,AA22LKF,PriceInUSD=4.99,4d9d5f8a-89ad-48a5-a1b6-7b8833b61ef3
7,2022-04-03 05:45:34,AA22LKF,PriceInUSD=1.99,936b493f-ced5-413c-9215-7352e59df041
8,2022-04-03 22:14:37,AA22LKF,PriceInUSD=4.99,a30b766c-2cb2-466a-ad38-31107ba12067
9,2022-04-01 21:35:04,AA22LKF,PriceInUSD=4.99,b566b000-1f43-49c6-a76d-914956b331e9


# **Schema Validation on both files**

In [ ]:


# Define the schema for the first dataset
leh_schema = {
    'date': is_datetime64_any_dtype,
    'userId': is_string_dtype,
    'country': is_string_dtype,
    'total_lp_earned': is_numeric_dtype
}

# Function to validate the dataset
def validate_dataset(dataset, schema):
    errors = []
    # Check for column count
    if len(dataset.columns) != len(schema):
        errors.append(f"Column count mismatch. Expected {len(schema)}, found {len(dataset.columns)}.")

    # Check for column names and types
    for column, check_function in schema.items():
        if column not in dataset.columns:
            errors.append(f"Missing expected column: {column}")
        else:
            if not check_function(dataset[column]):
                errors.append(f"Column '{column}' has incorrect data type. Expected {check_function.__name__}.")

    return errors

# Validate the first dataset
errors_first_dataset = validate_dataset(LEH_df, leh_schema)

# If there are errors, write them to a CSV file
if errors_first_dataset:
    error_log_path = '/content/error_sv_leh.csv'
    pd.DataFrame({'error': errors_first_dataset}).to_csv(error_log_path, index=False)
    result_message = f"Errors found. Details are written to {error_log_path}."
else:
    result_message = "No errors found. The dataset is valid according to the schema."

result_message


'No errors found. The dataset is valid according to the schema.'

In [ ]:

purchases_schema = {
    'date': is_datetime64_any_dtype,
    'userId': is_string_dtype,
    'revenue': is_string_dtype,
    'transaction_id': is_string_dtype
}

# Function to validate the dataset
def validate_dataset(dataset, schema):
    errors = []
    # Check for column count
    if len(dataset.columns) != len(schema):
        errors.append(f"Column count mismatch. Expected {len(schema)}, found {len(dataset.columns)}.")

    # Check for column names and types
    for column, check_function in schema.items():
        if column not in dataset.columns:
            errors.append(f"Missing expected column: {column}")
        else:
            if not check_function(dataset[column]):
                errors.append(f"Column '{column}' has incorrect data type. Expected {check_function.__name__}.")

    return errors

# Validate the first dataset
errors_first_dataset = validate_dataset(purchases_df, purchases_schema)

# If there are errors, write them to a CSV file
if errors_first_dataset:
    error_log_path = '/content/error_sv_purchases.csv'
    pd.DataFrame({'error': errors_first_dataset}).to_csv(error_log_path, index=False)
    result_message = f"Errors found. Details are written to {error_log_path}."
else:
    result_message = "No errors found. The dataset is valid according to the schema."

result_message


'No errors found. The dataset is valid according to the schema.'

# **Data Cleaning and Quality Check**

In [ ]:
def process_loyalty_earned_hourly(df):
    """
    Processes the Loyalty Earned Hourly data.

    Parameters:
    df (DataFrame): The input DataFrame containing the loyalty data.

    Returns:
    Tuple[str, str]: Paths to the processed data CSV file and errors CSV file.
    """
    processed_file_path = Path('/content/processed_LEH.csv')
    errors_file_path = Path('/content/errors_LEH.csv')

    try:
        # Separate records with missing values
        errors_df_list = [df[df.isnull().any(axis=1)].copy()]
        df.dropna(inplace=True)

        # Check for userId format
        user_id_pattern = r'^[A-Za-z]{2}\d{2}[A-Za-z]{3}$'
        invalid_user_ids = ~df['userId'].str.match(user_id_pattern)
        errors_df_list.append(df[invalid_user_ids].copy())
        df = df[~invalid_user_ids]

        # Validate country codes
        invalid_countries = ~df['country'].isin(['US', 'CA'])
        errors_df_list.append(df[invalid_countries].copy())
        df = df[~invalid_countries]

        # Convert total_lp_earned to numeric and check for negative values
        df['total_lp_earned'] = pd.to_numeric(df['total_lp_earned'], errors='coerce')
        invalid_lp_earned = df['total_lp_earned'] < 0
        errors_df_list.append(df[invalid_lp_earned].copy())
        df = df[~invalid_lp_earned]

        # Save the records that passed all checks
        df.to_csv(processed_file_path, index=False)

        # Combine all error records and save
        errors_df = pd.concat(errors_df_list, ignore_index=True)
        errors_df.to_csv(errors_file_path, index=False, mode='a', header=not Path(errors_file_path).exists())

    except Exception as e:
        print(f"An error occurred: {e}")
        # Handle the error or re-raise the exception
        # For example, you could return error messages or status codes

    return str(processed_file_path), str(errors_file_path)

# Example usage
processed_LEH_file_path, errors_LEH_file_path = process_loyalty_earned_hourly(LEH_df)


In [ ]:
def process_purchases(df):
    """
    Processes the Purchases data.

    Parameters:
    df (DataFrame): The input DataFrame containing the purchases data.

    Returns:
    Tuple[str, str]: Paths to the processed data CSV file and errors CSV file.
    """
    processed_file_path = Path('/content/processed_purchases.csv')
    errors_file_path = Path('/content/errors_purchases.csv')

    try:
        # Separate records with missing values
        errors_df_list = [df[df.isnull().any(axis=1)].copy()]
        df.dropna(inplace=True)

        # Ensure the dates are in the correct format like Loyalty Earned Hourly
        df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d %H:%M:%S')

        # Check for userId format
        user_id_pattern = r'^[A-Za-z]{2}\d{2}[A-Za-z]{3}$'
        invalid_user_ids = ~df['userId'].str.match(user_id_pattern)
        errors_df_list.append(df[invalid_user_ids].copy())
        df = df[~invalid_user_ids]

        # Extract and convert the revenue field to numeric
        df['revenue'] = df['revenue'].str.replace('PriceInUSD=', '', regex=False)
        df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
        # Append non-numeric and non-positive revenue records to errors
        errors_df_list.append(df[(df['revenue'].isnull()) | (df['revenue'] <= 0)].copy())
        # Keep only rows with positive revenue
        df = df[df['revenue'] > 0]

        # Check for duplicate records and remove them
        df = df.drop_duplicates(keep='first')

        # Save the records that passed all checks
        df.to_csv(processed_file_path, index=False)

        # Combine all error records and save
        errors_df = pd.concat(errors_df_list, ignore_index=True)
        errors_df.to_csv(errors_file_path, index=False, mode='a', header=not errors_file_path.exists())

    except Exception as e:
        print(f"An error occurred: {e}")
        # Handle the error or re-raise the exception
        # For example, you could return error messages or status codes


    return str(processed_file_path), str(errors_file_path)




In [ ]:
processed_purchases_file_path, errors_purchases_file_path = process_purchases(purchases_df)

# **Final Data Transformation and CSV generation**

In [ ]:
import pandas as pd

# Load the data
purchases_df = pd.read_csv('/content/processed_purchases.csv')
leh_df = pd.read_csv('/content/processed_LEH.csv')

# Convert 'date' to datetime and round down purchases to the nearest hour
purchases_df['date'] = pd.to_datetime(purchases_df['date'])
leh_df['date'] = pd.to_datetime(leh_df['date'])
purchases_df['date_hourly'] = purchases_df['date'].dt.floor('H')

# Merge the dataframes on 'userId' and 'date_hourly'
final_df = pd.merge(leh_df, purchases_df,  how='left', left_on=['userId','date'], right_on = ['userId','date_hourly'])

# Group by 'date_x' (hourly), 'userId', and 'country' for hourly summarization
hourly_summary = final_df.groupby(['date_x', 'userId', 'country']).agg(
    user_total_lp_earned=('total_lp_earned', 'sum'),
    user_total_revenue=('revenue', 'sum'),
    user_total_purchases=('transaction_id', 'count')
)
hourly_summary['user_avg_revenue_per_purchase'] = hourly_summary.apply(
    lambda x: x['user_total_revenue'] / x['user_total_purchases'] if x['user_total_purchases'] > 0 else 0, axis=1
)
hourly_summary.reset_index(inplace=True)

# Group by 'date_daily' and 'userId' for daily total revenue
final_df['date_daily'] = final_df['date_x'].dt.date
daily_revenue = final_df.groupby(['date_daily', 'userId']).agg(total_daily_revenue=('revenue', 'sum'))
daily_revenue.reset_index(inplace=True)
daily_revenue['date_daily'] = pd.to_datetime(daily_revenue['date_daily'])

# Merge daily revenue with hourly summary
hourly_summary['date_daily'] = pd.to_datetime(hourly_summary['date_x'].dt.date)
summary_table = pd.merge(hourly_summary, daily_revenue, how='left', on=['userId', 'date_daily'])

summary_table = summary_table.rename(columns={'date_x': 'app date'})
summary_table = summary_table.round(2)
# Save the summary table to a CSV file
summary_table.to_csv('summaryTable.csv', index=False)


In [ ]:
summary_table

,app date,userId,country,user_total_lp_earned,user_total_revenue,user_total_purchases,user_avg_revenue_per_purchase,date_daily,total_daily_revenue
0,2022-04-01 00:00:00,AA22LKF,US,95.99,0.00,0,0.00,2022-04-01,99.99
1,2022-04-01 01:00:00,AA22LKF,US,2.42,0.00,0,0.00,2022-04-01,99.99
2,2022-04-01 01:00:00,TT57LKF,US,5.31,1.99,1,1.99,2022-04-01,23.97
3,2022-04-01 04:00:00,WW42LKF,US,77.90,0.00,0,0.00,2022-04-01,0.00
4,2022-04-01 07:00:00,TT57LKF,US,73.12,1.99,1,1.99,2022-04-01,23.97
5,2022-04-01 10:00:00,TT57LKF,US,77.82,0.00,0,0.00,2022-04-01,23.97
6,2022-04-01 16:00:00,WW42LKF,US,93.29,0.00,0,0.00,2022-04-01,0.00
7,2022-04-01 20:00:00,AA22LKF,US,26.03,0.00,0,0.00,2022-04-01,99.99
8,2022-04-01 20:00:00,WW42LKF,CA,69.62,0.00,0,0.00,2022-04-01,0.00
9,2022-04-01 21:00:00,TT57LKF,US,16.46,0.00,0,0.00,2022-04-01,23.97
